# FORCE 11
## Exploring the data in SHARE

Introduction to SHARE Queries and the current state of SHARE Data

### Names are difficult to disambiguate

SHARE attempts to break names into Given, Family, and Additional pieces. The [SHARE person schema](https://github.com/CenterForOpenScience/SHARE-Schema/blob/master/share.yaml#L42) also includes spots for ```email```, ```affiliation```, and any links to other identifiers, such as ORCIDS, in the ```sameAs``` field.

Let's do a query to showcase different names appearing in the 5 most recent documents in SHARE

In [ ]:
import furl
import requests


def query_share(size, query=None):
    SHARE_API = 'https://osf.io/api/v1/share/search/'
    search_url = furl.furl(SHARE_API)
    search_url.args['size'] = size
    search_url.args['sort'] = 'providerUpdatedDateTime'
    if query:
        search_url.args['q'] = query
    return requests.get(search_url.url).json()

def print_title_contributors(results):
    for result in results['results']:
        print(result['title'].encode('utf-8'))
        print('~~~~~~~~~')
        for name in result['contributors']:
            print(name['name'])
        print('-------------------------------------------')
        
results =  query_share(5)
print_title_contributors(results)

#### Names don't always show up in the same format

Let's choose a name, remove the middle initial, and see if we get a result

In [ ]:
from sharepa import ShareSearch

def search_a_name(name):
    name_search = ShareSearch()
    name_search = name_search.query(
        {
            "bool": {
                "should": [
                    {
                        "match": {
                            "contributors.name": {
                                "query": name, 
                                "operator": "and",
                                "type" : "phrase"
                            }
                        }
                    }
                ]
            }
        }
    )
    
    return name_search

def print_name_results(name):
    search = search_a_name(name)
    if search.count() == 1:
        print('There is {} document with the contributor {}'.format(  search.count(), name))
    else:
        print('There are {} documents with the contributor {}'.format(search.count(), name))

print_name_results('Martone, Maryann E.')
print_name_results('Martone, Maryann')
print_name_results('Maryann Martone')
print_name_results('Maryann E Martone')

## Identifiers are Difficult to Identify

Here's a query that will search for all documents with contributors that have at least one orcid

In [ ]:
recent_results = query_share(3, 'contributors.sameAs:*orcid*')

print('There are {} results'.format(recent_results['count']))
print('----------')
for result in recent_results['results']:
    print(result['title'].encode('utf-8'))
    print(result['shareProperties']['source'])
    print('~~~~~~~~~')
    for name in result['contributors']:
        print('{} - {}'.format(name['name'].encode('utf-8'), name['sameAs']))
    print('-------------------------------------------')

Not every contributor on each document has an orcid suppplied.

### Try the same for another form of identifier - an email address

In [ ]:
# Contributors with Email Addresses

results = query_share(3, 'contributors.email:*')

print('There are {} results'.format(results['count']))
print('----------')
for result in results['results']:
    print(result['title'].encode('utf-8'))
    print(result['shareProperties']['source'])
    print('~~~~~~~~~')
    for name in result['contributors']:
            print('{} - {}'.format(name['name'].encode('utf-8'), name.get('email')))
    print('-------------------------------------------')


## No shared taxonomy for subjects or explicit document types - manuscripts, data, figures, etc.

One of our developers is working on this right now!

The DC field "Type" is an excellent place to start - however, there is a lot of variation for what is allowed inside of this field. 

"Element Description: The nature or genre of the content of the resource. Type includes terms describing general categories, functions, genres, or aggregation levels for content. Recommended best practice is to select a value from a controlled vocabulary (for example, the DCMIType vocabulary )."

In [ ]:
# Type

import pandas as pd
from sharepa import ShareSearch, basic_search
from sharepa.helpers import pretty_print

type_search = ShareSearch()
total_documents = basic_search.count()

type_search.aggs.bucket(
    'typeTermFilter',  # Every aggregation needs a name
    'terms',  # There are many kinds of aggregations
    field='otherProperties.properties.type',  # We store the source of a document in its type, so this will aggregate by source
    min_doc_count=1,
    exclude= "of|and|or",
    size=50,
)

type_results_executed = type_search.execute()

type_results = type_results_executed.aggregations.typeTermFilter.to_dict()['buckets']

type_dataframe = pd.DataFrame(type_results)
type_dataframe['percent'] = (type_dataframe['doc_count'] / total_documents)*100
type_dataframe

## Documents with Descriptions

Are abstracts copyrightable?

Abstracts, summaries and descriptions are not always made available. 

In [ ]:
# How many documents have descriptions?
from __future__ import division

results = query_share(10, 'description:*')

print('There are {} results'.format(results['count']))
print('{}/{} or {}% of results have descriptions'.format(results['count'], total_documents, (results['count']/total_documents)*100))